In [5]:
import itertools
import random
import math
from queue import Queue, PriorityQueue
from copy import deepcopy

# Name: Muhammad Abdullah
# Student ID: p22-9371
# Assignment#: Assignment no 2
# Section: BCS-6C

# Function to parse dataset from a file
def parse_dataset(file_path):
    data = {}
    print(f"Attempting to load: {file_path}")
    with open(file_path, 'r') as file:
        for line_num, line in enumerate(file, 1):
            line = line.strip()
            if not line:
                continue
            try:
                first_comma = line.index(',')
                last_comma = line.rindex(',')
                vertex = int(line[:first_comma])
                parent_set_str = line[first_comma + 1:last_comma]
                cost = float(line[last_comma + 1:])
                parent_set = set(eval(parent_set_str))
                parent_set = {int(p) for p in parent_set}
                if vertex not in data:
                    data[vertex] = []
                data[vertex].append((parent_set, cost))
            except Exception as e:
                print(f"Error in {file_path}, line {line_num}: {line} - {e}")
    vertices = sorted(data.keys())
    print(f"Loaded {len(vertices)} vertices from {file_path}: {vertices}")
    return data, vertices

# Load datasets
dataset_files = {
    "Dataset 0 (5 vertices)": "data0.txt",
    "Dataset 1 (18 vertices)": "data1.txt",
    "Dataset 2 (19 vertices)": "data2.txt",
    "Dataset 3 (19 vertices)": "data3.txt"
}

expected_counts = {
    "Dataset 0 (5 vertices)": 5,
    "Dataset 1 (18 vertices)": 18,
    "Dataset 2 (19 vertices)": 19,
    "Dataset 3 (19 vertices)": 19
}

datasets = {}
for name, file_path in dataset_files.items():
    try:
        data, vertices = parse_dataset(file_path)
        if len(vertices) != expected_counts[name]:
            print(f"Warning: {name} has {len(vertices)} vertices, expected {expected_counts[name]}.")
        datasets[name] = (data, vertices)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Skipping dataset '{name}'.")
    except Exception as e:
        print(f"Error parsing '{file_path}': {e}. Skipping dataset '{name}'.")

if not datasets:
    raise SystemExit("No datasets loaded.")

# Cost functions
def min_consistent_cost(vertex, ordering, data):
    pos = ordering.index(vertex)
    available = set(ordering[:pos])
    min_cost = float('inf')
    for parent_set, cost in data[vertex]:
        if parent_set.issubset(available):
            min_cost = min(min_cost, cost)
    return min_cost

def total_cost(ordering, data):
    return sum(min_consistent_cost(vertex, ordering, data) for vertex in ordering)

# 1. Hill Climbing
def hill_climbing(vertices, data):
    current = list(vertices)
    random.shuffle(current)
    current_cost = total_cost(current, data)
    while True:
        neighbors = []
        for i in range(len(current)):
            for j in range(i + 1, len(current)):
                neighbor = current.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        best_neighbor = min(neighbors, key=lambda x: total_cost(x, data), default=current)
        neighbor_cost = total_cost(best_neighbor, data)
        if neighbor_cost >= current_cost:
            break
        current = best_neighbor
        current_cost = neighbor_cost
    return tuple(current), current_cost

# 2. Simulated Annealing
def simulated_annealing(vertices, data, initial_temp=1000, cooling_rate=0.95):
    current = list(vertices)
    random.shuffle(current)
    current_cost = total_cost(current, data)
    temp = initial_temp
    while temp > 1:
        i, j = random.sample(range(len(current)), 2)
        neighbor = current.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        neighbor_cost = total_cost(neighbor, data)
        if neighbor_cost < current_cost or random.random() < math.exp((current_cost - neighbor_cost) / temp):
            current = neighbor
            current_cost = neighbor_cost
        temp *= cooling_rate
    return tuple(current), current_cost

# 3. BFS
def bfs_search(vertices, data):
    queue = Queue()
    queue.put([])
    best_ordering = None
    best_cost = float('inf')
    all_vertices = set(vertices)
    while not queue.empty():
        current = queue.get()
        if len(current) == len(vertices):
            cost = total_cost(current, data)
            if cost < best_cost:
                best_cost = cost
                best_ordering = tuple(current)
            continue
        remaining = all_vertices - set(current)
        for v in remaining:
            queue.put(current + [v])
    return best_ordering, best_cost

# 4. DFS (Brute Force)
def dfs_search(vertices, data):
    best_ordering = None
    best_cost = float('inf')
    for perm in itertools.permutations(vertices):
        cost = total_cost(perm, data)
        if cost < best_cost:
            best_cost = cost
            best_ordering = perm
    return best_ordering, best_cost

# 5. Minimax (Simplified)
def minimax_search(vertices, data):
    best_ordering = None
    best_cost = float('inf')
    for perm in itertools.permutations(vertices):
        cost = total_cost(perm, data)
        if cost < best_cost:
            best_cost = cost
            best_ordering = perm
    return best_ordering, best_cost

# 6. Greedy Best-First Search
def greedy_best_first_search(vertices, data):
    all_vertices = set(vertices)
    ordering = []
    while len(ordering) < len(vertices):
        remaining = all_vertices - set(ordering)
        best_vertex = min(remaining, key=lambda v: min_consistent_cost(v, ordering + [v], data))
        ordering.append(best_vertex)
    cost = total_cost(ordering, data)
    return tuple(ordering), cost

# 7. Optimized A* Search - Constraint-aware heuristic, lower cap
def a_star_search(vertices, data):
    def heuristic(partial, all_vertices, data):
        remaining = set(all_vertices) - set(partial)
        h_cost = 0
        placed = set(partial)
        for v in remaining:
            min_cost = float('inf')
            for parent_set, cost in data[v]:
                # Only count if all parents are either placed or still available
                if parent_set.issubset(placed) or parent_set.issubset(placed.union(remaining)):
                    min_cost = min(min_cost, cost)
            h_cost += min_cost if min_cost != float('inf') else 0  # Default to 0 if no valid option yet
        return h_cost

    pq = PriorityQueue()
    pq.put((0, [], 0))  # f_score, ordering, g_score
    all_vertices = set(vertices)
    state_count = 0
    max_states = 1000000  # Reduced cap, e.g., 1 million
    best_ordering = None
    best_cost = float('inf')

    while not pq.empty() and state_count < max_states:
        f_score, current, g_score = pq.get()
        state_count += 1

        if len(current) == len(vertices):
            if g_score < best_cost:
                best_cost = g_score
                best_ordering = tuple(current)
            continue

        remaining = all_vertices - set(current)
        for v in remaining:
            new_order = current + [v]
            g = total_cost(new_order, data)
            if g >= best_cost:  # Prune if already worse than best complete solution
                continue
            h = heuristic(new_order, all_vertices, data)
            f = g + h
            if f < best_cost:  # Only explore promising paths
                pq.put((f, new_order, g))

    if state_count >= max_states:
        print("A* Search hit state limit of 1,000,000; result may be suboptimal.")
    return best_ordering, best_cost if best_ordering else float('inf')

# 8. Uniform Cost Search - Kept for comparison
def uniform_cost_search(vertices, data):
    pq = PriorityQueue()
    pq.put((0, []))
    best_ordering = None
    best_cost = float('inf')
    all_vertices = set(vertices)
    state_count = 0
    max_states = 1000000  # Matching A* for fairness
    while not pq.empty() and state_count < max_states:
        cost_so_far, current = pq.get()
        state_count += 1
        if len(current) == len(vertices):
            if cost_so_far < best_cost:
                best_cost = cost_so_far
                best_ordering = tuple(current)
            continue
        remaining = all_vertices - set(current)
        for v in remaining:
            new_order = current + [v]
            new_cost = total_cost(new_order, data)
            pq.put((new_cost, new_order))
    if state_count >= max_states:
        print("Uniform Cost Search hit state limit of 1,000,000; result may be suboptimal.")
    return best_ordering, best_cost if best_ordering else float('inf')

# Main execution
searches = [
    ("Hill Climbing", hill_climbing),
    ("Simulated Annealing", simulated_annealing),
    ("BFS", bfs_search),
    ("DFS (Brute Force)", dfs_search),
    ("Minimax (Simplified)", minimax_search),
    ("Greedy Best-First Search", greedy_best_first_search),
    ("A* Search", a_star_search),
    ("Uniform Cost Search", uniform_cost_search)
]

for name, (data, vertices) in datasets.items():
    print(f"\nDataset: {name} ({len(vertices)} vertices)")
    print("-" * 50)
    if len(vertices) > 5:
        limited_searches = [(n, f) for n, f in searches if n not in ["DFS (Brute Force)", "BFS", "Minimax (Simplified)"]]
        print("Skipping DFS, BFS, and Minimax due to large size (factorial complexity).")
    else:
        limited_searches = searches

    for search_name, search_func in limited_searches:
        try:
            ordering, cost = search_func(vertices, data)
            print(f"{search_name}:")
            print(f"Ordering: {ordering}")
            print(f"Total Cost: {cost:.3f}\n")
        except Exception as e:
            print(f"Error in {search_name} for {name}: {e}\n")

Attempting to load: data0.txt
Loaded 5 vertices from data0.txt: [1, 2, 3, 4, 5]
Attempting to load: data1.txt
Loaded 18 vertices from data1.txt: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Attempting to load: data2.txt
Loaded 19 vertices from data2.txt: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Attempting to load: data3.txt
Loaded 19 vertices from data3.txt: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Dataset: Dataset 0 (5 vertices) (5 vertices)
--------------------------------------------------
Hill Climbing:
Ordering: (5, 3, 4, 1, 2)
Total Cost: 465.434

Simulated Annealing:
Ordering: (2, 3, 5, 4, 1)
Total Cost: 466.306

BFS:
Ordering: (4, 2, 5, 3, 1)
Total Cost: 465.434

DFS (Brute Force):
Ordering: (4, 2, 5, 3, 1)
Total Cost: 465.434

Minimax (Simplified):
Ordering: (4, 2, 5, 3, 1)
Total Cost: 465.434

Greedy Best-First Search:
Ordering: (2, 4, 5, 3, 1)
Total Cost: 465.435

A* Search:
Ordering: (4, 2, 5, 3, 1)
T